In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

In [3]:
def check_data(df):
    print(df.head())
    print("\n")
    print("SHAPE")
    print(df.shape)
    print("\n")
    print("DATA TYPES")
    print(df.dtypes)
    print("\n")
    print("UNIQUE VALUES")
    print(df.nunique())
    print("\n")
    print("MISSING VALUES")
    print(df.isna().sum())

In [4]:
response = requests.get("https://s02-infrastructure.s3.eu-west-1.amazonaws.com/hackathon-2/website.html")
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
main_data = soup.find_all('div', class_="main-data")

In [6]:
colnames = [colname.get_text() for colname in main_data[0].find_all("th")]

In [7]:
data = []
for row in main_data[0].find("tbody").find_all("tr"):
    data.append([cell.get_text() for cell in row.find_all("td")])

In [8]:
df = pd.DataFrame(data=data, columns=colnames)

In [9]:
missing_ids = soup.find_all('div', class_="missing-ids")
additional_ids = [x.get_text() for x in missing_ids[0].find("tbody").find_all("td")]

In [10]:
#checking if number of NA ids in main df is the same as length of additional ids
print(np.sum(df.id=="<NA>"), len(additional_ids))

999 999


In [11]:
for index, additional_id in enumerate(additional_ids):
    df.iloc[index].id=additional_id

In [12]:
#check if all ids have been successfully found
np.sum(df.id=="<NA>")

0

In [15]:
df = df.astype('float64')
df = df.rename(columns=str.lower)
df = df.set_index("id")
check_data(df)

        numdots  subdomainlevel  pathlevel  urllength  numdash  \
id                                                               
7368.0      1.0             0.0        3.0      116.0      0.0   
3881.0      4.0             1.0        2.0       44.0      0.0   
5393.0      2.0             0.0        2.0      108.0      0.0   
1401.0      3.0             1.0        2.0       40.0      0.0   
4925.0      2.0             1.0        3.0       42.0      0.0   

        numdashinhostname  atsymbol  tildesymbol  numunderscore  numpercent  \
id                                                                            
7368.0                0.0       0.0          0.0            3.0         0.0   
3881.0                0.0       0.0          0.0            0.0         0.0   
5393.0                0.0       0.0          0.0            2.0         0.0   
1401.0                0.0       0.0          0.0            0.0         0.0   
4925.0                0.0       0.0          1.0            0.0

In [16]:
df.to_csv("webscraping.csv")